# Preprocessing of Methylation Data
This notebook does some preprocessing for a script that takes annotation and downloaded methylation arrays and transforms it to one matrix $S \in \mathbb{R}^{p \times s}$ where $p$ is the number of genes and $s$ is the number of samples.

This matrix can then be used by yet another notebook to derive the final mean cancer type matrix.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn, h5py, os
from joblib import Parallel, delayed
%matplotlib inline

/home/sasse/.local/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
LOAD_ANNOTATION_WITH_PROM = False
m_file = '/project/gcn/diseasegcn/data/pancancer/TCGA/methylation/download/ff41e5fe-2564-4f23-942b-60fbdd33b596/jhu-usc.edu_BRCA.HumanMethylation27.2.lvl-3.TCGA-BH-A0HW-01A-11D-A032-05.gdc_hg38.txt'
m_file2 = '/project/gcn/diseasegcn/data/pancancer/TCGA/methylation/download/fff8ff71-7c4d-4f25-a611-7843b8b84b45/jhu-usc.edu_LUAD.HumanMethylation450.2.lvl-3.TCGA-35-5375-01A-01D-1626-05.gdc_hg38.txt'

## Load Methylation Array Data & Container
The methylation data gives us beta values per cpg sites. Each row corresponds to one cpg site. On the other hand, the annotation file gives us a list of genes and their genomic coordinates. It's wise to remove all the genes that we are not really interested in prior to calculating the mean beta values per gene because we then have to do less work.

In [3]:
m_df_1 = pd.read_csv(m_file, sep='\t')
m_df_1.dropna(axis=0, inplace=True)
m_df_2 = pd.read_csv(m_file2, sep='\t')
m_df_2.dropna(axis=0, inplace=True)

In [4]:
if LOAD_ANNOTATION_WITH_PROM:
    anno_file = '/project/gcn/diseasegcn/data/pancancer/TCGA/methylation/annotation_with_promoters_500bp.tsv'
    a_df = pd.read_csv(anno_file, sep='\t')
else:
    anno_file = '/project/gcn/diseasegcn/data/pancancer/TCGA/methylation/gencode.v28.annotation.gff3'
    a_df = pd.read_csv(anno_file, sep='\t', skiprows=7, header=None, names=['chr', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attr'])
    a_df.dropna(axis=0, inplace=True)
    a_df = a_df[a_df.type == 'gene']
    annotated_gene_ids = [i[0].strip().split('=')[1].strip().split('.')[0].strip() for i in a_df.attr.str.split(';')]
    annotated_gene_names = [i[3].strip().split('=')[1].strip() for i in a_df.attr.str.split(';')]
    a_df['ID'] = annotated_gene_ids
    a_df['Symbol'] = annotated_gene_names
    a_df.drop_duplicates(subset='Symbol', inplace=True) # remove duplicate genes (not interested in transcript level)
a_df.head()

,chr,source,type,start,end,score,strand,phase,attr,ID,Symbol
0,chr1,HAVANA,gene,11869.0,14409.0,.,+,.,ID=ENSG00000223972.5;gene_id=ENSG00000223972.5...,ENSG00000223972,DDX11L1
12,chr1,HAVANA,gene,14404.0,29570.0,.,-,.,ID=ENSG00000227232.5;gene_id=ENSG00000227232.5...,ENSG00000227232,WASH7P
25,chr1,ENSEMBL,gene,17369.0,17436.0,.,-,.,ID=ENSG00000278267.1;gene_id=ENSG00000278267.1...,ENSG00000278267,MIR6859-1
28,chr1,HAVANA,gene,29554.0,31109.0,.,+,.,ID=ENSG00000243485.5;gene_id=ENSG00000243485.5...,ENSG00000243485,RP11-34P13.3
36,chr1,ENSEMBL,gene,30366.0,30503.0,.,+,.,ID=ENSG00000284332.1;gene_id=ENSG00000284332.1...,ENSG00000284332,MIR1302-2


In [5]:
# load Container
data_file = '../../data/pancancer/hotnet_iref_vec_input_unbalanced.h5'

with h5py.File(data_file, 'r') as f:
    network = f['network'][:]
    features = f['features'][:]
    node_names = f['gene_names'][:]
    y_train = f['y_train'][:]
    y_test = f['y_test'][:]
    if 'y_val' in f:
        y_val = f['y_val'][:]
    else:
        y_val = None
    train_mask = f['mask_train'][:]
    test_mask = f['mask_test'][:]
    if 'mask_val' in f:
        val_mask = f['mask_val'][:]
    else:
        val_mask = None

# build data frame for node names
node_names_df = pd.DataFrame(node_names, columns=['ID', 'Symbol'])
print ("#genes in network but without annotation: {}".format(node_names_df[~node_names_df.Symbol.isin(a_df.Symbol)].shape[0]))
a_df = a_df[a_df.Symbol.isin(node_names_df.Symbol)]
print (a_df.shape)
a_df.drop_duplicates(subset='Symbol', inplace=True)

#genes in network but without annotation: 519


## Get Promoter Start and End

In [7]:
def promoter_window_wrapper(row):
    start, end = get_promotor_window(row['start'], row['end'], row['strand'])
    return start, end


def get_promotor_window(start, end, strand, meth_data=None):
    if strand == '+':
        tss = start
    else:
        tss = end
    if not meth_data is None: # fancy method to get promoter
        scan_region = (np.max(tss-1000, 0), tss + 1000)
        return calculate_promoter_window(scan_region[0], scan_region[1], meth_data)
    else:
        return np.max(tss-500, 0), tss + 500


def calculate_promoter_window(scan_start, scan_end, meth_data, shift=50, size=200):
    best_mean_meth = None
    best_window = (None, None)
    for i in range(int(scan_start), int(scan_end-size+1), shift):
        # get mean meth for window
        m_sites_window = meth_data[meth_data.Start.between(i, i+size)]
        m_mean_window = m_sites_window.Beta_value_mean.mean()
        
        # decide on best window
        if best_mean_meth is None: # first window
            best_mean_meth = m_mean_window
            best_window = i, i+size
        else: # any of the later windows
            if abs(best_mean_meth-m_mean_window) > 0.25: # large change in mean detected
                break
            else: # make the current window the best one
                best_mean_meth = m_mean_window
                best_window = i, i+size
    return best_window
promoter_vec = np.vectorize(get_promotor_window)

In [9]:
# get promoter start and end
x = a_df.apply(promoter_window_wrapper, axis=1)

# set it in the annotation dataframe and save that back to file
p_starts = np.array([i[0] for i in x])
p_ends = np.array([i[1] for i in x])
a_df['promoter_start'] = p_starts
a_df['promoter_end'] = p_ends
a_df.to_csv('../../data/pancancer/TCGA/methylation/annotation_with_promoters_1000bp.tsv', sep='\t')

# do a plot of the distribution
a_df['TSS'] = 0
a_df.loc[a_df.strand == '+', 'TSS'] = a_df[a_df.strand == '+'].promoter_start
a_df.loc[a_df.strand == '-', 'TSS'] = a_df[a_df.strand == '-'].promoter_end
d_to_prom = a_df.TSS - (p_ends+p_starts)/2
fig = plt.figure(figsize=(14, 8))
bins = np.linspace(-1000, 1000, 50)
plt.hist(d_to_prom, bins=bins)

## Get Experimental Strategy & Number of Samples

In [17]:
def get_filenames(meth_raw_dir):
    all_filenames = []
    for dname in os.listdir(meth_raw_dir):
        sub_dirname = os.path.join(meth_raw_dir, dname)
        if os.path.isdir(sub_dirname):
            for fname in os.listdir(sub_dirname):
                if fname.endswith('gdc_hg38.txt'):
                    all_filenames.append(os.path.join(sub_dirname, fname))
    return all_filenames

all_files = get_filenames('../../data/pancancer/TCGA/methylation/download/')

ctype_prots = {}
for f in all_files:
    fname = os.path.basename(f)
    protocol = fname.split('.')[2]
    cancer_type = fname.split('.')[1].split('_')[1].strip().lower()
    if not cancer_type in ctype_prots:
        ctype_prots[cancer_type] = [protocol]
    else:
        ctype_prots[cancer_type] = ctype_prots[cancer_type] + [protocol]

In [18]:
for c in ctype_prots:
    no_27k = len([i for i in ctype_prots[c] if i.endswith('27')])
    prop = no_27k / len(ctype_prots[c])
    print ("{} -> Total #Samples: {}\t#27k Arrays: {}\tProportion: {}".format(c, len(ctype_prots[c]), no_27k, prop))

laml -> Total #Samples: 280	#27k Arrays: 140	Proportion: 0.5
ucec -> Total #Samples: 603	#27k Arrays: 118	Proportion: 0.1956882255389718
hnsc -> Total #Samples: 580	#27k Arrays: 0	Proportion: 0.0
luad -> Total #Samples: 657	#27k Arrays: 150	Proportion: 0.228310502283105
kirc -> Total #Samples: 899	#27k Arrays: 414	Proportion: 0.46051167964404893
ov -> Total #Samples: 623	#27k Arrays: 613	Proportion: 0.9839486356340289
gbm -> Total #Samples: 450	#27k Arrays: 295	Proportion: 0.6555555555555556
brca -> Total #Samples: 1234	#27k Arrays: 342	Proportion: 0.2771474878444084
lusc -> Total #Samples: 573	#27k Arrays: 161	Proportion: 0.28097731239092494
blca -> Total #Samples: 440	#27k Arrays: 0	Proportion: 0.0
coad -> Total #Samples: 556	#27k Arrays: 203	Proportion: 0.36510791366906475
read -> Total #Samples: 179	#27k Arrays: 73	Proportion: 0.40782122905027934
